In [1]:
!pip install langchain_core langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 194.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.0 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.17 which is incompatible.


In [ ]:
# 导入必要的库
import os
import json

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import Runnable # 用于链式调用

# 导入文档加载器 (如果需要从文件加载)
# from langchain_community.document_loaders import PyPDFLoader, TextLoader
# from langchain_text_splitters import RecursiveCharacterTextSplitter


# --- 配置部分 ---

# 确保你已经设置了 GOOGLE_API_KEY 环境变量
# 命令行设置示例 (Linux/macOS): export GOOGLE_API_KEY="你的API密钥"
# 命令行设置示例 (Windows): set GOOGLE_API_KEY="你的API密钥"
# 或者在脚本中直接设置 (不推荐，安全性原因):
# os.environ["GOOGLE_API_KEY"] = ""

if "GOOGLE_API_KEY" not in os.environ:
    raise ValueError("请设置 GOOGLE_API_KEY 环境变量。")

# 选择一个适合的 Gemini 模型
# gemini-1.5-pro-latest 拥有更大的上下文窗口，适合处理长文档
# gemini-1.0-pro 更常见，成本可能更低
# 请确保你的 API Key 有权限访问所选模型
GEMINI_MODEL = "gemini-1.5-pro-latest"
TEMPERATURE = 0.7 # 控制模型输出的随机性，0.0 表示确定性最高，1.0 最高


# --- 教学材料输入 ---

# 你可以直接在这里粘贴文本材料
# 或者实现从文件读取的逻辑 (见下面的注释示例)
teaching_material = """
## 细胞简介

细胞是生命的基本单位。所有已知的生物体都由细胞组成。有些生物体是单细胞的（如细菌和酵母），而其他生物体是多细胞的（如植物和动物）。

细胞通常非常小，只能在显微镜下观察到。细胞的结构复杂，包含多种细胞器，这些细胞器执行特定的功能。

### 细胞的组成部分

所有细胞都共享一些基本组成部分：
1. **细胞膜**: 包围细胞，控制物质进出。
2. **细胞质**: 细胞膜内的果冻状物质，包含细胞器。
3. **遗传物质**: 包含细胞生长、发育和复制所需的指令 (DNA)。

### 原核细胞与真核细胞

细胞可以分为两类：原核细胞和真核细胞。
* **原核细胞**: 没有细胞核，遗传物质位于细胞质的核区。例子：细菌。结构相对简单。
* **真核细胞**: 具有真正的细胞核，遗传物质被核膜包围。具有膜结合细胞器（如线粒体、叶绿体）。例子：植物细胞、动物细胞、真菌细胞、原生生物。结构复杂。

### 细胞器的例子 (真核细胞)

* **细胞核**: 包含DNA，控制细胞活动。
* **线粒体**: 能量生成（呼吸作用）的场所。
* **叶绿体**: 光合作用的场所（仅植物细胞和部分原生生物）。
* **内质网**: 合成蛋白质和脂质。
* **高尔基体**: 修改、分类和包装蛋白质和脂质。
* **液泡**: 存储物质，维持细胞膨胀（植物细胞通常有一个大液泡）。
* **溶酶体**: 包含消化酶，分解废物和细胞器（主要在动物细胞）。

### 总结

细胞是生命的基础，尽管大小和结构各异，但都执行生命的基本功能。理解细胞结构对于理解生命至关重要。
"""

# --- 从文件加载材料的示例 (取消注释并使用) ---
# def load_material_from_file(file_path):
#     """根据文件扩展名加载教学材料"""
#     file_extension = os.path.splitext(file_path)[1].lower()
#     documents = []
#     if file_extension == '.pdf':
#         loader = PyPDFLoader(file_path)
#         documents = loader.load()
#     elif file_extension == '.txt':
#         loader = TextLoader(file_path)
#         documents = loader.load()
#     # 添加更多文件类型支持...
#     else:
#         raise ValueError(f"不支持的文件类型: {file_extension}")

#     # 对于长文档，可能需要分割文本
#     # text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
#     # split_docs = text_splitter.split_documents(documents)

#     # 将所有文档内容拼接成一个字符串给LLM处理 (简单方式)
#     # 注意: 这可能会超出LLM的上下文窗口限制，对于大文件需要更复杂的处理策略 (如MapReduce)
#     material_text_from_file = "\n".join([doc.page_content for doc in documents])
#     return material_text_from_file

# # 使用文件加载 (示例，请替换 'your_material.pdf' 为实际文件路径)
# # try:
# #     teaching_material = load_material_from_file("your_material.pdf")
# # except FileNotFoundError:
# #     print("错误: 未找到文件。请检查文件路径。")
# #     exit()
# # except ValueError as e:
# #     print(f"错误: {e}")
# #     exit()
# # except Exception as e:
# #     print(f"加载文件时发生意外错误: {e}")
# #     exit()

# --- Langchain 处理部分 ---

# 1. 初始化 Gemini LLM
llm = ChatGoogleGenerativeAI(model=GEMINI_MODEL, temperature=TEMPERATURE)

# 2. 定义 Prompt 模板
# 这个模板指示 LLM 根据输入的文本生成 PPT 大纲，并要求严格输出 JSON 格式。
# 明确要求不要包含额外的文本或 markdown 标记（如 ```json```）。
prompt_template = """
你是一名专业的教育助手，请根据以下教学材料，为教师生成一个用于课堂教学的PPT大纲。

请按照以下JSON格式严格输出结果，不要包含任何额外的文字、markdown标记（如```json```）或其他说明：
{{
    "slides": [
        {{
            "slide_title": "...",
            "slide_bullets": ["...", "..."],
            "teacher_notes": "...", // 可选: 教师备课或讲解时可以参考的额外信息
            "suggested_visual": "..." // 可选: 建议的图片或图表类型
        }},
        ... // 多个幻灯片对象
    ]
}}

请确保幻灯片内容清晰、简洁，适合课堂讲解，并且逻辑流畅。要点概括应准确反映材料内容。
每张幻灯片的要点 (slide_bullets) 应该是列表形式，包含关键信息。

教学材料内容：
{material_text}
"""

prompt = ChatPromptTemplate.from_template(prompt_template)

# 3. 初始化 JSON 输出解析器
# Langchain 的 JsonOutputParser 会尝试解析 LLM 的字符串输出为 Python 字典/列表
output_parser = JsonOutputParser()

# 4. 构建 Langchain Chain
# 使用 LCEL (Langchain Expression Language) 将各个组件连接起来:
# prompt -> llm -> output_parser
chain = (
    prompt
    | llm
    | output_parser
)

# --- 执行和输出 ---

print(f"正在使用 Gemini 模型 '{GEMINI_MODEL}' 生成 PPT 大纲...")
print("-" * 40)

try:
    # 调用链，传入教学材料文本
    # chain.invoke() 会执行整个流程：格式化prompt -> 调用LLM -> 解析输出
    ppt_outline_json_structure = chain.invoke({"material_text": teaching_material})

    # 打印生成的 PPT 大纲 (Python 字典/列表)
    print("\n--- 生成的 PPT 大纲 (JSON 结构) ---")
    # 使用 json.dumps 格式化输出，使其更易读，并处理中文显示
    print(json.dumps(ppt_outline_json_structure, indent=2, ensure_ascii=False))

    # 你现在可以将 ppt_outline_json_structure 这个 Python 对象
    # 传递给一个使用 python-pptx 的函数来创建实际的 .pptx 文件

except Exception as e:
    print(f"\n发生错误: {e}")
    print("请检查:")
    print("1. 是否正确设置了 GOOGLE_API_KEY 环境变量。")
    print(f"2. 你的 API Key 是否有权限访问模型 '{GEMINI_MODEL}'。")
    print("3. 输入材料是否过长，超出了模型的上下文窗口限制。")
    print("4. LLM 返回的格式是否符合预期的 JSON 结构 (可以尝试调整Prompt)。")

print("\n" + "-" * 40)
print("大纲生成完成。下一步是将这个 JSON 结构转换为实际的 .pptx 文件。")




正在使用 Gemini 模型 'gemini-1.5-pro-latest' 生成 PPT 大纲...
----------------------------------------

--- 生成的 PPT 大纲 (JSON 结构) ---
{
  "slides": [
    {
      "slide_title": "细胞：生命的基本单位",
      "slide_bullets": [
        "所有生物体都由细胞组成",
        "单细胞生物体 vs. 多细胞生物体",
        "细胞通常微小，需要显微镜观察",
        "复杂结构，包含多种细胞器"
      ],
      "teacher_notes": "从宏观到微观引入细胞的概念，强调细胞是生命的基本单位。可以准备一些单细胞和多细胞生物的图片。",
      "suggested_visual": "不同生物体（细菌、植物、动物）的图片"
    },
    {
      "slide_title": "细胞的基本组成部分",
      "slide_bullets": [
        "细胞膜：控制物质进出",
        "细胞质：果冻状物质，包含细胞器",
        "遗传物质 (DNA)：控制细胞生长、发育和复制"
      ],
      "teacher_notes": "可以将细胞比作一个工厂，细胞膜是围墙，细胞质是厂房，DNA是控制中心。",
      "suggested_visual": "细胞结构简图，突出细胞膜、细胞质和DNA"
    },
    {
      "slide_title": "原核细胞 vs. 真核细胞",
      "slide_bullets": [
        "原核细胞：无细胞核，结构简单，例如细菌",
        "真核细胞：有细胞核，结构复杂，例如动植物细胞"
      ],
      "teacher_notes": "强调原核和真核细胞的主要区别在于细胞核的有无以及结构的复杂程度。",
      "suggested_visual": "原核细胞和真核细胞的结构对比图"
    },
    {
      "slide_title": "真核细